In [42]:
import pandas as pd
df=pd.read_csv("word2bec.csv")
import numpy as np
import time

In [2]:
df.head()

,Unnamed: 0,0,0.1,1,2,3,4,5,6,7,...,40,41,42,43,44,45,46,47,48,49
0,0,5,-0.008193,0.329279,0.625574,0.496134,-0.333714,0.289197,-0.747551,-0.244537,...,0.224683,0.270239,-0.287705,-0.401288,0.115455,-0.007333,-0.036545,0.338190,-0.247735,0.002495
1,1,2,0.378093,-0.187072,-0.040618,0.570556,-0.071080,0.267960,-0.267498,-0.104090,...,0.646922,0.665386,-0.029825,-0.047749,0.219141,-0.276389,-0.107161,-0.109785,0.665813,-0.319745
2,2,13,0.425933,-0.147378,0.346750,0.788827,-0.256113,0.000148,-0.780406,0.268517,...,0.698619,0.141133,-0.439271,-0.273757,-0.339629,-0.228902,-0.007452,0.052214,0.351986,-0.166502
3,3,11,0.305797,-0.114674,-0.112830,1.366427,-0.265462,0.334894,-0.473779,0.472661,...,0.989978,0.069972,-0.337342,-0.162852,-0.034660,-0.668402,-0.068475,0.269458,0.271057,-0.260155
4,4,3,0.049649,0.102235,-0.428460,1.526369,-0.187836,0.693541,-0.376602,0.601414,...,1.113309,-0.240243,-0.405946,-0.173767,0.344461,-0.559076,0.012242,0.605369,0.052500,-0.335685


# データ分離

In [10]:
df_Y=df["0"]
df_X=df.iloc[:,2:]

In [12]:
df_X.head()

,0.1,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.008193,0.329279,0.625574,0.496134,-0.333714,0.289197,-0.747551,-0.244537,-0.151849,0.144963,...,0.224683,0.270239,-0.287705,-0.401288,0.115455,-0.007333,-0.036545,0.338190,-0.247735,0.002495
1,0.378093,-0.187072,-0.040618,0.570556,-0.071080,0.267960,-0.267498,-0.104090,-0.244015,0.091265,...,0.646922,0.665386,-0.029825,-0.047749,0.219141,-0.276389,-0.107161,-0.109785,0.665813,-0.319745
2,0.425933,-0.147378,0.346750,0.788827,-0.256113,0.000148,-0.780406,0.268517,-0.558237,0.138638,...,0.698619,0.141133,-0.439271,-0.273757,-0.339629,-0.228902,-0.007452,0.052214,0.351986,-0.166502
3,0.305797,-0.114674,-0.112830,1.366427,-0.265462,0.334894,-0.473779,0.472661,-0.600808,0.977719,...,0.989978,0.069972,-0.337342,-0.162852,-0.034660,-0.668402,-0.068475,0.269458,0.271057,-0.260155
4,0.049649,0.102235,-0.428460,1.526369,-0.187836,0.693541,-0.376602,0.601414,-0.610354,1.619612,...,1.113309,-0.240243,-0.405946,-0.173767,0.344461,-0.559076,0.012242,0.605369,0.052500,-0.335685


In [79]:
#arrayバージョン
np_X_train=np.array(df_X[:31987])
np_Y_train=np.array(df_Y[:31987])
np_X_test=np.array(df_X[31988:])
np_Y_test=np.array(df_Y[31988:])

In [25]:
#dataframeバージョン
df_X_train=df_X[:31987]
df_Y_train=df_Y[:31987]
df_X_test=df_X[31988:]
df_Y_test=df_Y[31988:]

In [78]:
df_X_train.shape

(31987, 50)

# ランダムフォレスト

In [100]:
from sklearn.ensemble import RandomForestClassifier
t1 = time.time() 

rfc_1 = RandomForestClassifier(random_state=0, n_estimators=10)
rfc_1.fit(df_X_train, df_Y_train)

t2 = time.time()
# 経過時間を表示
elapsed_time = t2-t1
print("経過時間：{}".format(elapsed_time))

経過時間：3.349816083908081


# 勾配ブースティング

In [52]:
from sklearn.ensemble import GradientBoostingClassifier
t1 = time.time() 

gbct = GradientBoostingClassifier(random_state=0, max_depth=3, learning_rate=0.1)
gbct.fit(df_X_train, df_Y_train)

t2 = time.time()
# 経過時間を表示
elapsed_time = t2-t1
print("経過時間：{}".format(elapsed_time))

経過時間：303.18718004226685


# lightGBM

In [119]:
import lightgbm as lgb
t1 = time.time() 
lgb_train = lgb.Dataset(np_X_train, np_Y_train)
lgb_eval = lgb.Dataset(np_X_test, np_Y_test, reference=lgb_train)
lgbm_params = {'objective': 'multiclass',"num_class":15}#多値分類＆クラスは１５
model1 = lgb.train(lgbm_params, lgb_train, valid_sets=lgb_eval)
t2 = time.time()
# 経過時間を表示
elapsed_time = t2-t1
print("経過時間：{}".format(elapsed_time))

[1]	valid_0's multi_logloss: 2.37133
[2]	valid_0's multi_logloss: 2.2141
[3]	valid_0's multi_logloss: 2.09407
[4]	valid_0's multi_logloss: 1.99639
[5]	valid_0's multi_logloss: 1.91495
[6]	valid_0's multi_logloss: 1.84448
[7]	valid_0's multi_logloss: 1.78451
[8]	valid_0's multi_logloss: 1.73158
[9]	valid_0's multi_logloss: 1.68494
[10]	valid_0's multi_logloss: 1.64249
[11]	valid_0's multi_logloss: 1.60517
[12]	valid_0's multi_logloss: 1.57125
[13]	valid_0's multi_logloss: 1.54075
[14]	valid_0's multi_logloss: 1.51324
[15]	valid_0's multi_logloss: 1.48787
[16]	valid_0's multi_logloss: 1.46453
[17]	valid_0's multi_logloss: 1.44345
[18]	valid_0's multi_logloss: 1.42405
[19]	valid_0's multi_logloss: 1.40625
[20]	valid_0's multi_logloss: 1.38959
[21]	valid_0's multi_logloss: 1.37424
[22]	valid_0's multi_logloss: 1.36079
[23]	valid_0's multi_logloss: 1.34796
[24]	valid_0's multi_logloss: 1.3355
[25]	valid_0's multi_logloss: 1.32453
[26]	valid_0's multi_logloss: 1.31442
[27]	valid_0's multi_lo

# SVM

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaler.fit(df_X_train)
X_train_std = scaler.transform(df_X_train)

scaler.fit(df_X_test)
X_test_std = scaler.transform(df_X_test)

In [43]:
from sklearn.svm import SVC
t1 = time.time() 
rbf_svm = SVC(kernel='linear', gamma=0.1, C=10)
rbf_svm.fit(df_X_train, df_Y_train)
t2 = time.time()
# 経過時間を表示
elapsed_time = t2-t1
print("経過時間：{}".format(elapsed_time))

経過時間：238.69311094284058


# NN

In [47]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

Using TensorFlow backend.


In [114]:
model = Sequential()  
model.add(Dense(900,input_shape=(50,)))
model.add(Activation('relu'))
model.add(Dense(1000))
model.add(Activation('relu'))
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dense(15))
model.add(Activation('softmax'))

In [115]:
from keras.utils import np_utils  
y_train_one_hot = np_utils.to_categorical(df_Y_train, num_classes=15).astype('int') 
y_test_one_hot = np_utils.to_categorical(df_Y_test, num_classes=15).astype('int')
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])    

t1 = time.time()
hist = model.fit(df_X_train,
                 y_train_one_hot,
                 epochs=10,
                 validation_data=(df_X_test, y_test_one_hot),
                 verbose=1,
                batch_size=100)
t2 = time.time()
elapsed_time = t2-t1
print("経過時間：{}".format(elapsed_time))

Train on 31987 samples, validate on 13707 samples
Epoch 1/10
31987/31987 [==============================] - 11s 349us/step - loss: 2.0711 - acc: 0.3553 - val_loss: 1.7867 - val_acc: 0.4390
Epoch 2/10
31987/31987 [==============================] - 11s 335us/step - loss: 1.6627 - acc: 0.4968 - val_loss: 1.5435 - val_acc: 0.5235
Epoch 3/10
31987/31987 [==============================] - 11s 356us/step - loss: 1.5095 - acc: 0.5359 - val_loss: 1.4473 - val_acc: 0.5562
Epoch 4/10
31987/31987 [==============================] - 11s 337us/step - loss: 1.4332 - acc: 0.5589 - val_loss: 1.3860 - val_acc: 0.5753
Epoch 5/10
31987/31987 [==============================] - 11s 356us/step - loss: 1.3869 - acc: 0.5722 - val_loss: 1.3495 - val_acc: 0.5871
Epoch 6/10
31987/31987 [==============================] - 11s 332us/step - loss: 1.3543 - acc: 0.5807 - val_loss: 1.3234 - val_acc: 0.5936
Epoch 7/10
31987/31987 [==============================] - 11s 330us/step - loss: 1.3296 - acc: 0.5894 - val_loss: 1.

# ロジスティック

In [140]:
from sklearn.linear_model import LogisticRegression
t1 = time.time()
clf = LogisticRegression()
clf.fit(df_X_train, df_Y_train)
t2 = time.time()
elapsed_time = t2-t1
print("経過時間：{}".format(elapsed_time))

経過時間：21.399709939956665


# ランダムフォレスト

In [60]:
from sklearn.metrics import accuracy_score
print('精度（訓練）:',accuracy_score(df_Y_train, rfc_1.predict(df_X_train)))
print('精度（検証）:',accuracy_score(df_Y_test, rfc_1.predict(df_X_test)))

精度（訓練）: 0.984056022759246
精度（検証）: 0.591522579703801


# 勾配ブースティング

In [61]:
print('精度（訓練）:',accuracy_score(df_Y_train, gbct.predict(df_X_train)))
print('精度（検証）:',accuracy_score(df_Y_test, gbct.predict(df_X_test)))

精度（訓練）: 0.6839028355269328
精度（検証）: 0.6201940614284672


# SVM

In [82]:
print('精度（訓練）:',accuracy_score(df_Y_train, rbf_svm.predict(df_X_train)))
print('精度（検証）:',accuracy_score(df_Y_test, rbf_svm.predict(df_X_test)))

精度（訓練）: 0.6545784224841342
精度（検証）: 0.6547019770919968


# NN

In [117]:
print('精度（訓練）:',accuracy_score(df_Y_train,np.argmax(model.predict(df_X_train),axis=1)))
print('精度（検証）:',accuracy_score(df_Y_test, np.argmax(model.predict(df_X_test),axis=1)))

精度（訓練）: 0.6029011786038078
精度（検証）: 0.6131903407018312


# LightGBM

In [129]:
y_pred= model1.predict(np_X_train)
y_pred_max=np.argmax(y_pred,axis=1)
accuracy1=sum(np_Y_train==y_pred_max)/len(np_Y_train)
print("訓練:{}".format(accuracy1))
y_pred= model1.predict(np_X_test)
y_pred_max=np.argmax(y_pred,axis=1)
accuracy2=sum(np_Y_test==y_pred_max)/len(np_Y_test)
print("検証：{}".format(accuracy2))

訓練:0.8097352049270016
検証：0.6448529948201649


# ロジスティック回帰

In [139]:
print('精度（訓練）:',accuracy_score(df_Y_train, clf.predict(df_X_train)))
print('精度（検証）:',accuracy_score(df_Y_test, clf.predict(df_X_test)))

精度（訓練）: 0.6299746772126176
精度（検証）: 0.6371926752754067
